In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import findspark
findspark.init()

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").appName("Spark MLib Intro").getOrCreate()
spark

In [4]:
emp_df = spark.read.csv( "./data/employee_salary.csv", header=True, inferSchema=True )
emp_df.show()

+---------+---+----------+------+
|     name|age|experience|salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [6]:
emp_df.printSchema()

root
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- experience: integer (nullable = true)
 |-- salary: integer (nullable = true)



In [15]:
emp_df.dtypes

[('name', 'string'), ('age', 'int'), ('experience', 'int'), ('salary', 'int')]

In [16]:
emp_df.columns

['name', 'age', 'experience', 'salary']

- For spark MLib, to perform and apply different machine learning algorithms on any dataset, we have a specific kind of data processing, in which we need to collate all the data columns that forms the independent feature, in following way: \
`[independent_feature_1, independent_feature_2, ...., independent_feature_n]` \
This group of independent features is called a **VectorAssembler**(a class in python). This group then becomes a new, different, independent feature.
- In our case, the independent features are `age` and `experience`, and they are grouped as below: \
  `["age", "experience"]`
- Every record is grouped like this.

In [17]:
from pyspark.ml.feature import VectorAssembler

In [18]:
independent_feature_group = VectorAssembler( inputCols=["age", "experience"], outputCol="Independent Features" )

In [19]:
print(type(independent_feature_group))

<class 'pyspark.ml.feature.VectorAssembler'>


In [20]:
output_df = independent_feature_group.transform( emp_df )

In [21]:
output_df.show()

+---------+---+----------+------+--------------------+
|     name|age|experience|salary|Independent Features|
+---------+---+----------+------+--------------------+
|    Krish| 31|        10| 30000|         [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|          [30.0,8.0]|
|    Sunny| 29|         4| 20000|          [29.0,4.0]|
|     Paul| 24|         3| 20000|          [24.0,3.0]|
|   Harsha| 21|         1| 15000|          [21.0,1.0]|
|  Shubham| 23|         2| 18000|          [23.0,2.0]|
+---------+---+----------+------+--------------------+



In [22]:
final_df = output_df.select( ["Independent Features", "salary"] )
final_df.show()

+--------------------+------+
|Independent Features|salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



In [23]:
print( type(final_df) )

<class 'pyspark.sql.dataframe.DataFrame'>


In [24]:
# Training data and testing data split
train_data_df, test_data_df = final_df.randomSplit([0.75, 0.25])
print( type(train_data_df) )
print( type(test_data_df) )

<class 'pyspark.sql.dataframe.DataFrame'>
<class 'pyspark.sql.dataframe.DataFrame'>


In [25]:
train_data_df.show()
test_data_df.show()

+--------------------+------+
|Independent Features|salary|
+--------------------+------+
|          [21.0,1.0]| 15000|
|          [24.0,3.0]| 20000|
|          [29.0,4.0]| 20000|
|          [30.0,8.0]| 25000|
+--------------------+------+

+--------------------+------+
|Independent Features|salary|
+--------------------+------+
|          [23.0,2.0]| 18000|
|         [31.0,10.0]| 30000|
+--------------------+------+



In [26]:
from pyspark.ml.regression import LinearRegression

In [27]:
salary_regressor = LinearRegression( featuresCol="Independent Features", labelCol="salary" )
salary_regressor = salary_regressor.fit( train_data_df )

In [31]:
# coefficients
salary_regressor.coefficients

DenseVector([47.619, 1285.7143])

In [32]:
# intercept
salary_regressor.intercept

13619.047619047662

In [33]:
# Predictions
pred_results = salary_regressor.evaluate( test_data_df )

In [36]:
print(type(pred_results))
print(type( pred_results.predictions ))

<class 'pyspark.ml.regression.LinearRegressionSummary'>
<class 'pyspark.sql.dataframe.DataFrame'>


In [37]:
pred_results.predictions.show()

+--------------------+------+-----------------+
|Independent Features|salary|       prediction|
+--------------------+------+-----------------+
|          [23.0,2.0]| 18000|17285.71428571428|
|         [31.0,10.0]| 30000|27952.38095238097|
+--------------------+------+-----------------+



In [38]:
pred_results.meanAbsoluteError

1380.9523809523762

In [39]:
pred_results.rootMeanSquaredError

1533.4516369623352